<a href="https://colab.research.google.com/github/fadel11-hub/WORKSHOP-MSIB-5/blob/main/WORKSHOP_COMPUTER_VISION_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Convolutional Neural Network(CNN)

## Import library

In [1]:
import tensorflow as tf
import matplotlib as plt
import os

## Prepare dataset

In [2]:
import zipfile
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

--2024-01-28 07:25:44--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.207, 74.125.195.207, 172.253.117.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   237MB/s    in 0.6s    

2024-01-28 07:25:44 (237 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]



In [3]:
# Get the Horse or Human validation dataset
!wget -q -P /content/ https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

val_local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(val_local_zip, 'r')
zip_ref.extractall('validation-horse-or-human')
zip_ref.close()


In [4]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('./horse-or-human/humans')

# Directory with validation horse pictures
validation_horses_dir = os.path.join('validation-horse-or-human/horses')
# Directory with validation human pictures
validation_humans_dir = os.path.join('validation-horse-or-human/humans')

In [5]:
base_dir = '/content/'

print("Contents of base directory:")
print(os.listdir(base_dir))

print("\nContents of train directory:")
print(os.listdir(f'{base_dir}/horse-or-human'))

print("\nContents of validation directory:")
print(os.listdir(f'{base_dir}/validation-horse-or-human'))

Contents of base directory:
['.config', 'horse-or-human.zip', 'horse-or-human', 'validation-horse-or-human', 'validation-horse-or-human.zip', 'sample_data']

Contents of train directory:
['humans', 'horses']

Contents of validation directory:
['humans', 'horses']


In [6]:
print('total training horses images :', len(os.listdir(  train_horse_dir ) ))
print('total training humans images :', len(os.listdir(  train_human_dir ) ))

print('total validation horses images :', len(os.listdir( validation_horses_dir ) ))
print('total validation humans images :', len(os.listdir( validation_humans_dir ) ))

total training horses images : 500
total training humans images : 527
total validation horses images : 128
total validation humans images : 128


## Image preprocessing

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255)



ImageDataGenerator.flow_from_directory() adalah metode yang digunakan untuk menghasilkan generator data dari direktori.


*   ./horse-or-human/ adalah direktori yang berisi data latih.
*   target_size=(300, 300) adalah ukuran gambar yang diinginkan.
*   batch_size=128 adalah jumlah gambar yang akan dimuat dalam satu batch.
*   class_mode='binary' adalah mode kelas untuk data latih.

Dalam kasus ini, data latih memiliki dua kelas, yaitu kuda dan manusia.





In [8]:
train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  # direktori data latih
        target_size=(300, 300),  # mengubah resolusi seluruh gambar menjadi 300X300 piksel
        batch_size=128,
        class_mode='binary' # karena ini merupakan masalah klasifikasi 2 kelas, gunakan class_mode = 'binary'
        )

validation_generator = train_datagen.flow_from_directory(
        './validation-horse-or-human/', # direktori data validasi
        target_size=(300, 300), # mengubah resolusi seluruh gambar menjadi 3000x300 piksel
        batch_size=20,
        class_mode='binary',# karena ini merupakan masalah klasifikasi 2 kelas gunakan class_mode = 'binary'
        )

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## Fungsi dari Code ini
1. print(train_generator.class_indices)

Baris ini menampilkan kamus yang memetakan nama kelas ke indeks numerik yang digunakan oleh generator data untuk mewakili setiap kelas.
Contoh output:
{'horses': 0, 'humans': 1}
Pada output ini, kelas "horses" memiliki indeks 0, dan kelas "humans" memiliki indeks 1.

2. labels = '\n'.join(sorted(train_generator.class_indices.keys()))

Baris ini membuat sebuah string yang berisi nama-nama kelas, dipisahkan oleh newline (\n).
Prosesnya:
Mengambil nama-nama kelas dari kunci dalam kamus train_generator.class_indices.
Mengurutkan nama-nama kelas secara alfabetis.
Menggabungkan nama-nama kelas yang sudah diurutkan dengan newline sebagai pemisah.



In [9]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

# with open('/content/drive/MyDrive/Bangkit_Academy/WOKRHOP MSIB 5/Dataset/label.txt', 'w') as f:
  # f.write(labels)

{'horses': 0, 'humans': 1}


## Build Model

 **Berikut penjelasan kode tersebut secara detail dalam bahasa Indonesia:**

**1. `model = tf.keras.models.Sequential(...)`**

- Baris ini membuat model baru bernama `model` dengan struktur Sequential, artinya lapisan model disusun secara berurutan.

**2. `tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3))`**

- Lapisan Convolutional 2D pertama dengan:
    - 16 filter (kernel) berukuran 3x3
    - Fungsi aktivasi ReLU (Rectified Linear Unit)
    - Input shape sebesar 300x300 piksel dengan 3 saluran warna (RGB)

**3. `tf.keras.layers.MaxPooling2D(2,2)`**

- Lapisan Max Pooling 2D dengan ukuran pool 2x2 untuk mengurangi dimensi spasial dan menangkap fitur-fitur penting.

**dan seterusnya sampai dengan lapisan Flatten**

**10. `tf.keras.layers.Flatten()`**

- Lapisan Flatten untuk mengubah output tensor 3D dari lapisan Convolutional menjadi tensor 1D (vektor) agar dapat diproses oleh lapisan Dense.

**11. `tf.keras.layers.Dense(64, activation='relu')`**

- Lapisan Dense dengan 64 neuron dan aktivasi ReLU untuk mempelajari representasi fitur yang lebih kompleks.

**12. `tf.keras.layers.Dense(1, activation='sigmoid')`**

- Lapisan Dense output dengan 1 neuron dan aktivasi sigmoid untuk menghasilkan probabilitas antara 0 dan 1, cocok untuk masalah klasifikasi biner (dua kelas).

**Kesimpulan:**

- Kode ini mendefinisikan model klasifikasi gambar berbasis CNN (Convolutional Neural Network) yang terdiri dari beberapa lapisan Convolutional dan Max Pooling, diikuti oleh lapisan Dense untuk klasifikasi akhir.
- Model ini akan belajar mengekstraksi fitur-fitur penting dari gambar untuk membedakan antara dua kelas.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 35, 35, 64)        0

 **Berikut penjelasan kode tersebut secara detail:**

**1. `model.compile(...)`**

- Baris ini mempersiapkan model untuk pelatihan dengan menentukan fungsi loss, optimizer, dan metrik yang akan digunakan.

**2. `loss='binary_crossentropy'`**

- Menentukan fungsi loss yang mengukur perbedaan antara prediksi model dan label sebenarnya untuk setiap contoh. Dalam kasus klasifikasi biner, `binary_crossentropy` adalah fungsi loss yang umum digunakan.

**3. `optimizer=tf.optimizers.RMSprop(learning_rate=0.001)`**

- Menentukan algoritma optimizer yang digunakan untuk memperbarui bobot model selama pelatihan.
- `tf.optimizers.RMSprop` adalah optimizer yang sering digunakan untuk model deep learning.
- `learning_rate=0.001` mengatur seberapa besar perubahan yang akan dilakukan pada bobot model pada setiap langkah pelatihan.

**4. `metrics=['accuracy']`**

- Menentukan metrik yang akan digunakan untuk mengevaluasi performa model selama pelatihan dan validasi.
- Dalam kasus ini, `accuracy` (akurasi) digunakan untuk mengukur persentase prediksi yang benar.


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
epochs=7
# latih model dengan model.fit
history = model.fit(
            train_generator,
            epochs=epochs, # tambahkan epochs jika akurasi model belum optimal
            validation_data=validation_generator # menampilkan akurasi pengujian data validasi
)

Epoch 1/7
9/9 [==============================] - 18s 962ms/step - loss: 0.8049 - accuracy: 0.4800 - val_loss: 0.6909 - val_accuracy: 0.5000
Epoch 2/7
9/9 [==============================] - 9s 1s/step - loss: 0.6741 - accuracy: 0.5531 - val_loss: 0.6361 - val_accuracy: 0.5586
Epoch 3/7
9/9 [==============================] - 9s 1s/step - loss: 0.6659 - accuracy: 0.6154 - val_loss: 0.6219 - val_accuracy: 0.6484
Epoch 4/7
9/9 [==============================] - 9s 959ms/step - loss: 0.5633 - accuracy: 0.7478 - val_loss: 0.7062 - val_accuracy: 0.6836
Epoch 5/7
9/9 [==============================] - 10s 987ms/step - loss: 0.5807 - accuracy: 0.8423 - val_loss: 0.8355 - val_accuracy: 0.8008
Epoch 6/7
9/9 [==============================] - 9s 1s/step - loss: 0.1784 - accuracy: 0.9435 - val_loss: 1.4920 - val_accuracy: 0.7734
Epoch 7/7
9/9 [==============================] - 9s 1s/step - loss: 0.1692 - accuracy: 0.9279 - val_loss: 2.6377 - val_accuracy: 0.6797


## Testing

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():

  # Memprediksi Image
  path = '/content/' + fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])

  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")


# Tranfer learning & Augmentation

## Image Preprocessing

** Penjelasan kode ImageDataGenerator :**

**1. `train_datagen_2 = ImageDataGenerator(...)`**

- Baris ini membuat sebuah objek ImageDataGenerator bernama `train_datagen_2` yang akan digunakan untuk menghasilkan versi gambar yang telah dimodifikasi untuk meningkatkan variasi dalam dataset Anda.

**2. `rescale=1./255`**

- Mengubah nilai piksel gambar dari rentang 0-255 menjadi 0-1. Ini penting karena model pembelajaran mesin seringkali bekerja lebih baik dengan nilai input yang dinormalisasi.

**3. `rotation_range=20`**

- Merotasi gambar secara acak hingga 20 derajat ke kiri atau kanan.

**4. `horizontal_flip=True`**

- Membalik gambar secara horizontal secara acak.

**5. `shear_range=0.2`**

- Memiringkan gambar secara acak hingga 20% secara horizontal.

**Kesimpulan:**

- Kode ini menyiapkan generator data yang akan menghasilkan versi gambar yang telah dimodifikasi secara acak untuk meningkatkan variasi dalam dataset Anda.
- Modifikasi ini dapat membantu model pembelajaran mesin untuk menjadi lebih robust (tahan) terhadap perubahan-perubahan kecil pada gambar dan untuk menggeneralisasi lebih baik pada data baru.
- Pemisahan set validasi juga penting untuk memastikan model tidak mengalami overfitting (terlalu menyesuaikan dengan data latih).


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen_2 = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    )

validation_datagen_2 = ImageDataGenerator(rescale=1./255)



In [ ]:
train_generatort_aug = train_datagen_2.flow_from_directory(
        './horse-or-human/',  # direktori data latih
        target_size=(300, 300),
        batch_size=4,
        class_mode='binary',
        )

validation_generatortf = validation_datagen_2.flow_from_directory(
        './validation-horse-or-human/', # direktori data validasi
        target_size=(300, 300),
        batch_size=4,
        class_mode='binary',
        )

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## Model Transfer Learning

1. base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

Baris ini memuat model MobileNetV2 yang sudah terlatih sebelumnya dari TensorFlow Hub.
Penjelasan:
* tf.keras.applications.MobileNetV2: fungsi untuk memuat model MobileNetV2.
input_shape=IMG_SHAPE: menentukan ukuran gambar input yang sesuai dengan yang telah kita definisikan sebelumnya.

* include_top=False: menghilangkan lapisan teratas dari model, yang biasanya digunakan untuk klasifikasi 1000 kelas ImageNet. Kita akan menambahkan lapisan kita sendiri untuk tugas klasifikasi spesifik yang kita perlukan.

* weights='imagenet': menggunakan bobot yang telah dilatih sebelumnya pada dataset ImageNet yang besar, berisi jutaan gambar. Bobot ini sudah mengandung pengetahuan umum tentang fitur-fitur visual yang dapat membantu dalam berbagai tugas klasifikasi gambar.

2. base_model.trainable = False

* Baris ini menetapkan model MobileNetV2 menjadi tidak dapat dilatih. Artinya, selama pelatihan model, bobot dalam model MobileNetV2 tidak akan diubah. Hal ini dilakukan untuk mempercepat pelatihan dan mencegah overfitting, karena model dasar sudah memiliki pengetahuan yang cukup baik tentang fitur-fitur visual

In [ ]:
IMG_SHAPE = (300, 300, 3)

# Buat model dasar dari MobileNet V2 yang telah dilatih sebelumnya
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False,
                                              weights='imagenet')
base_model.trainable = False

In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 300, 300, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 150, 150, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 150, 150, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 150, 150, 32)         0         ['bn_Conv1[

In [ ]:
model_aug = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# compile model dengan 'RMSprop' optimizer loss function 'binary_crossentropy'
model_aug.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
result = model_aug.fit(train_generatort_aug,
                      validation_data=validation_generator,
                      epochs=3)

Epoch 1/3
257/257 [==============================] - 36s 126ms/step - loss: 0.3048 - accuracy: 0.9883 - val_loss: 0.0054 - val_accuracy: 0.9961
Epoch 2/3
257/257 [==============================] - 28s 111ms/step - loss: 0.0656 - accuracy: 0.9961 - val_loss: 1.3375e-05 - val_accuracy: 1.0000
Epoch 3/3
257/257 [==============================] - 30s 116ms/step - loss: 0.0508 - accuracy: 0.9990 - val_loss: 0.0249 - val_accuracy: 0.9961


## Testing

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():

  path = '/content/' + fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])

  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")


Saving 20211210132813_IMG_3716.JPG to 20211210132813_IMG_3716.JPG
1/1 [==============================] - 0s 19ms/step
[0.9998436]
20211210132813_IMG_3716.JPG is a human
